# Here qe load all jsons and all excels and create a unqiue csv file per market

In [1]:
import os, re
import pandas as pd
import json

In [2]:
allyears = ['../datasets/Dataset_Mayy2019', 
            '../datasets/Dataset_July2020', 
            '../datasets/Dataset_Apri2021']
excelpaths = ['../datasets/Dataset_Mayy2019/Dataset_Mayy2019_traceability.csv',
              '../datasets/Dataset_July2020/Dataset_july2020_traceability.csv',
              '../datasets/Dataset_Apri2021/Dataset_April2021_traceability.csv',
             ]

## Auxiliar fuctions

In [3]:
import os,re
def getFiles(dirName):
    listOfFile = os.listdir(dirName)
    completeFileList = list()
    for file in listOfFile:
        completePath = os.path.join(dirName, file)
        if os.path.isdir(completePath):
            completeFileList = completeFileList + getFiles(completePath)
        elif 'file.json' in completePath:
            completeFileList.append(completePath.replace('\\', '/'))

    return completeFileList

#load all objects in the json files in a dictoinary with key = filepath
import json
def loadObjects(jsonPath):
    lines = open(jsonPath, 'r').readlines() 
    allobj = []
    try:
        for line in lines: 
            if(len(line.strip()) > 0):
                allobj.append( json.loads(line) )
    except:
        pass
    return allobj

def _createSkillDict(oldd, market, year):
    '''
    example json obj
    {'Main Category': 'Business & Finance', 'SubCategory': 'Business & Finance', 'Sample_Invocation_Utterances': ["“'Alexa, Open Current Bitcoin'”", "“'Alexa, Ask Current Bitcoin for Prices'”", "“'Alexa, Ask Current Bitcoin what the price is'”"], 'Name': 'CurrentBitcoin', 'Developer': 'by Sarim Studios', 'Skill_permission': [], 'Account_linking': '', 'Review_Count': None, 'Rating': '', 'Total_customer_that_rate_the_skill': '0', 'Total_Customers_Reviews': 'No customer reviews', 'Cost': 'Free to Enable', 'In_skill_purchase': None, 'Skill_description': "Description\nAn 'easy to use' Alexa skill that will provide you with the latest price for a bitcoin. This skill uses BitStamp's latest prices, which are always up-to-date. All you have to do to begin using the skill is ask Alexa, Ask Current Bitcoin for Prices. Price is given in USD.", 'Skill_link': 'https://www.amazon.ca/Sarim-Studios-CurrentBitcoin/dp/B01N9SS2LI/ref=sr_1_635?dchild=1&fst=as%3Aoff&qid=1595709096&rnid=16286270011&s=alexa-skills&sr=1-635', 'privacy_policy and Terms_of_use specified': 'No privacy_policy and Terms_of_use specified'}
    '''
    newd = {}    
    for obj in oldd:
        if len(obj) == 0:
            continue
            
        name = ''
        if 'Name' in obj:
            name = obj['Name'].strip()
        else:
            continue
            
        inskill = ''
        if 'In_skill_purchase' in obj:
            inskill = obj['In_skill_purchase']
            if(inskill is not None):
                inskill = inskill.strip()
        
        linking = ''
        if 'Account_linking' in obj:
            linking = obj['Account_linking']
            if(linking is not None):
                linking = linking.strip()
            
        dev = ''
        if 'Developer' in obj:
            dev = obj['Developer']
            if(dev is not None):
                dev = dev.strip()
        else:
            continue

        total_rev=''
        if 'Total_Customers_Reviews' in obj:
            total_rev = obj['Total_Customers_Reviews']
            if(total_rev is not None):
                total_rev = total_rev.strip()
        else:
            continue
        
        skill_id = ''
        if 'Skill_link' in obj:
            try:
                skill_url = obj['Skill_link']
                skill_id = re.search(r"\b[B]+[0-9]+(?:[A-Z0-9]+/)+", skill_url).group().replace('/','')
                if(skill_id is not None):
                    skill_id = skill_id.strip()
            except:
                pass
        
        maincat = ''
        if 'Main Category' in obj:
            maincat = obj['Main Category']
            if(maincat is not None):
                maincat = maincat.strip()
        
        cost = ''
        if 'Cost' in obj:
            cost = obj['Cost']
            if(cost is not None):
                cost = cost.strip()
            
        subcat = ''
        if 'SubCategory' in obj:
            subcat = obj['SubCategory']
            if(subcat is not None):
                subcat = subcat.strip()
        
        perm = obj['Skill_permission']

        
        skill_link = ''
        if 'Skill_link' in obj:
            skill_link = obj['Skill_link']
            if(skill_link is not None):
                skill_link = skill_link.strip()
        
        policy = ''
        if 'privacy_policy' in obj:
            policy = obj['privacy_policy']
            
        #we use id_aux as a unique identifier between yearly datasets
        id_aux = name+'__'+ dev +'__'+ skill_id
        
        newd[id_aux] = {
             'id_name_dev'  : name+'__'+ dev,
             'id_aux'  :id_aux,
             'id'      :skill_id,
             'name'    :name,
             'dev'     :dev,
             'cat'     :maincat,
             'market'  :market,
             'subcat'  :subcat,
             'acc_linking':linking,
             'in_skill_purchase':inskill,
             'cost':cost,
             'review_stars':total_rev,
             'skill_link':skill_link,
             'policy':policy,
             'policy_url':None,
             'policy_text':None,
             'perm_requested_norm':None,
             'perm_found_norm':None,
             'perm_requested_original':None,
             'traceability':None,
             'year':year}
         
        
        
        
    return newd

In [4]:
import pandas as pd

def reloadData(path):
    def getFiles(dirName):
        listOfFile = os.listdir(dirName)
        completeFileList = list()
        for file in listOfFile:
            completePath = os.path.join(dirName, file)
            if os.path.isdir(completePath):
                completeFileList = completeFileList + getFiles(completePath)
            elif '.json' in completePath:
                completeFileList.append(completePath.replace('\\', '/'))

        return completeFileList

    allf = getFiles( path )
    print(allf)

    #create dictionary of markets and skills
    skillsmarketd = {}
    for f in allf:
        marketname = f.split('/')[4].split('.json')[0]
        with open(f, 'r') as json_file:
            skillsmarketd[marketname] = json.load(json_file)

    print()
    print('Total unique skills per market')
    for key,value in skillsmarketd.items():
        print(key, ": ", len(value))
    print()
    print()
    return skillsmarketd

def loadExcelFile(path):
    df = pd.read_csv(path)
    df['unique_id'] = df['Name']+'__'+df['Developer']
    return df

def updateDataframe(skillsobject, t_df):
    '''
    skillsobject : object as returned by reloadData function
    t_df : tracability dataframe, as returned by loadExcelFile
    '''
    
    #first transform the disctionary of objects into a list of objects :)
    finallist = []
    for market in skillsobject:
        skills = skillsobject[market]
        for skill in skills:
            finallist.append( skills[skill] )
    df_aux = pd.DataFrame([x for x in finallist])
    
    #we need to fill in the blanks for #policy, policy_url, policy_text, 
    # perm_requested, perm_found, perm_requested_original, traceability with t_df
    t_df.rename(columns = {'unique_id':'id_name_dev'}, inplace = True)
    t_df = t_df.drop(['Developer', 'Name', 'acc_linking', 'cat',
           'in_skill_purchase',
           'subcat'], axis = 1)

    df_aux = df_aux.drop(['traceability','policy_url', 'policy_text', 'perm_requested_original', 'perm_found_norm',
       'perm_requested_norm'], axis = 1)
    mergedDf = df_aux.merge(t_df,how='left', on='id_name_dev')

   
    mergedDf.rename(columns = {'privacy_policy':'policy_url'}, inplace = True)
    mergedDf.rename(columns = {'privacy_policy_doc':'policy_text'}, inplace = True)
    mergedDf.rename(columns = {'Skill_permission':'perm_requested_original'}, inplace = True)
    mergedDf.rename(columns = {'perm_found':'perm_found_norm'}, inplace = True)
    mergedDf.rename(columns = {'permissions_requested_adapt':'perm_requested_norm'}, inplace = True)
    mergedDf = mergedDf.drop(['policy'], axis = 1)
        
   
    
    return mergedDf

# 1. Load datasets, clean duplicates, create processed dataset, load processed dataset and map traceabiulity

## 1.1 2019

### Load info from the 2019 json file

In [5]:
allf = getFiles( allyears[0] )
print(allf)

#NOTICE allo might contain repetitoins! (we filter them later)
allo = {}
for f in allf:
    allo[f] = loadObjects(f)
#list all markets
for key,value in allo.items():
    print(key)
    print(len(value))
    
#filter repetitoins and skills without name or developer

import json
year = '2019'

#Create a dictionary of unqiue skills per market
for f in allf:
    print('********************')
    market_name = f.split('/')[3]
    print(market_name)
    skillsdict = _createSkillDict(allo[f], market_name, year)
    print('#skills', len(skillsdict))
    
    skillsfound = []
    for sk in skillsdict:
        skill  = skillsdict[sk]
        skname = skill['name']
        skdev = skill['dev']      
     
    #save analysis in NewFullDataset json
    path = '../datasets/DatasetsProcessed/'+year+'/'+market_name+'.json'
    with open(path, 'w') as json_file:
        print('Saving json in ',path)
        json.dump(skillsdict, json_file)   

['../datasets/Dataset_Mayy2019/AU/file.json', '../datasets/Dataset_Mayy2019/CA/file.json', '../datasets/Dataset_Mayy2019/DE/file.json', '../datasets/Dataset_Mayy2019/ES/file.json', '../datasets/Dataset_Mayy2019/FR/file.json', '../datasets/Dataset_Mayy2019/IN/file.json', '../datasets/Dataset_Mayy2019/IT/file.json', '../datasets/Dataset_Mayy2019/JP/file.json', '../datasets/Dataset_Mayy2019/MX/file.json', '../datasets/Dataset_Mayy2019/UK/file.json', '../datasets/Dataset_Mayy2019/US/file.json']
../datasets/Dataset_Mayy2019/AU/file.json
59311
../datasets/Dataset_Mayy2019/CA/file.json
156204
../datasets/Dataset_Mayy2019/DE/file.json
57521
../datasets/Dataset_Mayy2019/ES/file.json
60147
../datasets/Dataset_Mayy2019/FR/file.json
30010
../datasets/Dataset_Mayy2019/IN/file.json
92770
../datasets/Dataset_Mayy2019/IT/file.json
45458
../datasets/Dataset_Mayy2019/JP/file.json
78583
../datasets/Dataset_Mayy2019/MX/file.json
26438
../datasets/Dataset_Mayy2019/UK/file.json
159644
../datasets/Dataset_Ma

### 1.1.2 Load filtered Dataset for 2019 and map skill traceability from excel File

In [6]:
#from excel
#test the join of the dataframes
year = '2019'
print('loading', excelpaths[0])
df2019 = loadExcelFile( excelpaths[0] )

#from json
datasetpath = "../datasets/DatasetsProcessed/"+year+"/"
skillsmarket = reloadData(datasetpath)

#join the two tables to create a general dataframe
finaldf = updateDataframe(skillsmarket, df2019)

#save dataframe
finaldf.to_csv('../datasets/DatasetsProcessedTraceability/2019.csv')

loading ../datasets/Dataset_Mayy2019/Dataset_Mayy2019_traceability.csv
['../datasets/DatasetsProcessed/2019/AU.json', '../datasets/DatasetsProcessed/2019/CA.json', '../datasets/DatasetsProcessed/2019/DE.json', '../datasets/DatasetsProcessed/2019/ES.json', '../datasets/DatasetsProcessed/2019/FR.json', '../datasets/DatasetsProcessed/2019/IN.json', '../datasets/DatasetsProcessed/2019/IT.json', '../datasets/DatasetsProcessed/2019/JP.json', '../datasets/DatasetsProcessed/2019/MX.json', '../datasets/DatasetsProcessed/2019/UK.json', '../datasets/DatasetsProcessed/2019/US.json']

Total unique skills per market
AU :  23123
CA :  24700
DE :  8928
ES :  1286
FR :  1341
IN :  20989
IT :  2210
JP :  2679
MX :  897
UK :  29094
US :  51338




### 1.1.3 Some checks on the final dataset for 2019

In [8]:
finaldf = pd.read_csv('../datasets/DatasetsProcessedTraceability/2019.csv')

In [8]:
finaldf.columns

Index(['Unnamed: 0', 'id_new', 'id_aux', 'id', 'name', 'dev', 'cat', 'market',
       'subcat', 'acc_linking', 'in_skill_purchase', 'cost', 'review_stars',
       'skill_link', 'year', 'perm_requested_original', 'perm_found_norm',
       'perm_requested_norm', 'policy_url', 'policy_text', 'traceability'],
      dtype='object')

#### Check total number of skills

In [29]:
finaldf.describe()

,Unnamed: 0,id,in_skill_purchase,skill_link,year
count,166585.000000,0.0,0.0,0.0,166585.0
mean,83292.000000,NaN,NaN,NaN,2019.0
std,48089.091634,NaN,NaN,NaN,0.0
min,0.000000,NaN,NaN,NaN,2019.0
25%,41646.000000,NaN,NaN,NaN,2019.0
50%,83292.000000,NaN,NaN,NaN,2019.0
75%,124938.000000,NaN,NaN,NaN,2019.0
max,166584.000000,NaN,NaN,NaN,2019.0


In [9]:
allmarkets = finaldf.market.unique()
for mk in allmarkets:
    print('{} unique skills found in {}'.format(len(finaldf.loc[finaldf['market'] == mk] ), mk))

23123 unique skills found in AU
24700 unique skills found in CA
8928 unique skills found in DE
1286 unique skills found in ES
1341 unique skills found in FR
20989 unique skills found in IN
2210 unique skills found in IT
2679 unique skills found in JP
897 unique skills found in MX
29094 unique skills found in UK
51338 unique skills found in US


#### Check how many of these skills have a policy

In [31]:
allmarkets = finaldf.market.unique()
for mk in allmarkets:
    auxdf = finaldf.loc[ (finaldf['market']==mk) & (finaldf['perm_requested_norm'].notnull())]
    print('{} unique skills have a policy in {}'.format(len(auxdf), mk))

120 unique skills have a policy in AU
158 unique skills have a policy in CA
0 unique skills have a policy in DE
0 unique skills have a policy in ES
0 unique skills have a policy in FR
111 unique skills have a policy in IN
0 unique skills have a policy in IT
0 unique skills have a policy in JP
0 unique skills have a policy in MX
287 unique skills have a policy in UK
658 unique skills have a policy in US


#### Check number of unique developers per market

In [32]:
allmarkets = finaldf.market.unique()
for mk in allmarkets:
    marketdf = finaldf.loc[finaldf['market'] == mk]
    alldevs = marketdf.dev.unique()
    print('{} unique devs in {}'.format(len(alldevs), mk))

10123 unique devs in AU
10773 unique devs in CA
3165 unique devs in DE
716 unique devs in ES
641 unique devs in FR
9197 unique devs in IN
1095 unique devs in IT
1056 unique devs in JP
540 unique devs in MX
12078 unique devs in UK
19507 unique devs in US


#### Broken, partial, complete skills per market

In [33]:
allmarkets = finaldf.market.unique()
totals = {'B':0, 'C':0, 'P':0}
for mk in allmarkets:
    print('>Market', mk)
    marketdf = finaldf.loc[finaldf['market'] == mk]
    alltypestraceability = marketdf.traceability.unique()
    print(alltypestraceability)
    for trace in alltypestraceability:        
        tracdf = marketdf.loc[marketdf['traceability'] == trace]
        if(tracdf is not None and len(tracdf)>0):
            totals[trace]+=len(tracdf)
            print('\t {} unique skills of type {}'.format(len(tracdf), trace))

>Market AU
[nan 'B' 'C' 'P']
	 52 unique skills of type B
	 59 unique skills of type C
	 9 unique skills of type P
>Market CA
[nan 'C' 'P' 'B']
	 85 unique skills of type C
	 17 unique skills of type P
	 56 unique skills of type B
>Market DE
[nan]
>Market ES
[nan]
>Market FR
[nan]
>Market IN
[nan 'C' 'B' 'P']
	 55 unique skills of type C
	 43 unique skills of type B
	 13 unique skills of type P
>Market IT
[nan]
>Market JP
[nan]
>Market MX
[nan]
>Market UK
[nan 'B' 'C' 'P']
	 109 unique skills of type B
	 152 unique skills of type C
	 26 unique skills of type P
>Market US
[nan 'P' 'C' 'B']
	 69 unique skills of type P
	 337 unique skills of type C
	 252 unique skills of type B


#### check how many skills have the same name__dev

In [34]:
allmarkets = finaldf.market.unique()
for mk in allmarkets:
    marketdf = finaldf.loc[finaldf['market'] == mk]
    print('{} market, {} unique ids, {} total entries'.format(mk, len(marketdf.id_aux.unique()), len(marketdf) ))
    

AU market, 23123 unique ids, 23123 total entries
CA market, 24700 unique ids, 24700 total entries
DE market, 8928 unique ids, 8928 total entries
ES market, 1286 unique ids, 1286 total entries
FR market, 1341 unique ids, 1341 total entries
IN market, 20989 unique ids, 20989 total entries
IT market, 2210 unique ids, 2210 total entries
JP market, 2679 unique ids, 2679 total entries
MX market, 897 unique ids, 897 total entries
UK market, 29094 unique ids, 29094 total entries
US market, 51338 unique ids, 51338 total entries


## 1.2 2020

### Load info from the 2020 json file

In [7]:
allf = getFiles( allyears[1] )
print(allf)

#NOTICE allo might contain repetitoins! (we filter them later)
allo = {}
for f in allf:
    allo[f] = loadObjects(f)
#list all markets
for key,value in allo.items():
    print(key)
    print(len(value))
    
#filter repetitoins and skills without name or developer
import json
year = '2020'

#Create a dictionary of unqiue skills per market
for f in allf:
    print('********************')
    market_name = f.split('/')[3]
    print(market_name)
    skillsdict = _createSkillDict(allo[f], market_name, year)
    print('#skills', len(skillsdict))
    
    skillsfound = []
    for sk in skillsdict:
        skill  = skillsdict[sk]
        skname = skill['name']
        skdev = skill['dev']      
     
    #save analysis in NewFullDataset json
    path = '../datasets/DatasetsProcessed/'+year+'/'+market_name+'.json'
    with open(path, 'w') as json_file:
        print('Saving json in ',path)
        json.dump(skillsdict, json_file) 

['../datasets/Dataset_July2020/AU/file.json', '../datasets/Dataset_July2020/CA/file.json', '../datasets/Dataset_July2020/DE/file.json', '../datasets/Dataset_July2020/ES/file.json', '../datasets/Dataset_July2020/FR/file.json', '../datasets/Dataset_July2020/IN/file.json', '../datasets/Dataset_July2020/IT/file.json', '../datasets/Dataset_July2020/JP/file.json', '../datasets/Dataset_July2020/MX/file.json', '../datasets/Dataset_July2020/UK/file.json', '../datasets/Dataset_July2020/US/file.json']
../datasets/Dataset_July2020/AU/file.json
68244
../datasets/Dataset_July2020/CA/file.json
42429
../datasets/Dataset_July2020/DE/file.json
53177
../datasets/Dataset_July2020/ES/file.json
49001
../datasets/Dataset_July2020/FR/file.json
2575
../datasets/Dataset_July2020/IN/file.json
133209
../datasets/Dataset_July2020/IT/file.json
5271
../datasets/Dataset_July2020/JP/file.json
46451
../datasets/Dataset_July2020/MX/file.json
2308
../datasets/Dataset_July2020/UK/file.json
41450
../datasets/Dataset_July20

### Create final dataset w/ traceability

In [8]:
#from excel
#test the join of the dataframes
year = '2020'
print('loading', excelpaths[1])
df2019 = loadExcelFile( excelpaths[1] )

#from json
datasetpath = "../datasets/DatasetsProcessed/"+year+"/"
skillsmarket = reloadData(datasetpath)

#join the two tables to create a general dataframe
finaldf = updateDataframe(skillsmarket, df2019)

#save dataframe
finaldf.to_csv('../datasets/DatasetsProcessedTraceability/'+year+'.csv')

loading ../datasets/Dataset_July2020/Dataset_july2020_traceability.csv
['../datasets/DatasetsProcessed/2020/AU.json', '../datasets/DatasetsProcessed/2020/CA.json', '../datasets/DatasetsProcessed/2020/DE.json', '../datasets/DatasetsProcessed/2020/ES.json', '../datasets/DatasetsProcessed/2020/FR.json', '../datasets/DatasetsProcessed/2020/IN.json', '../datasets/DatasetsProcessed/2020/IT.json', '../datasets/DatasetsProcessed/2020/JP.json', '../datasets/DatasetsProcessed/2020/MX.json', '../datasets/DatasetsProcessed/2020/UK.json', '../datasets/DatasetsProcessed/2020/US.json']

Total unique skills per market
AU :  24062
CA :  26027
DE :  10287
ES :  5010
FR :  2288
IN :  31246
IT :  4203
JP :  3545
MX :  1972
UK :  34618
US :  55736




In [13]:
finaldf.columns

Index(['id_new', 'id_aux', 'id', 'name', 'dev', 'cat', 'market', 'subcat',
       'acc_linking', 'in_skill_purchase', 'cost', 'review_stars',
       'skill_link', 'year', 'perm_requested_original', 'perm_found_norm',
       'policy_url', 'policy_text', 'traceability', 'perm_requested_norm'],
      dtype='object')

### Some checks and analyses

In [14]:
finaldf = pd.read_csv('../datasets/DatasetsProcessedTraceability/2020.csv')

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (9,10,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### total numer of skills

In [15]:
allmarkets = finaldf.market.unique()
for mk in allmarkets:
    print('{} unique skills found in {}'.format(len(finaldf.loc[finaldf['market'] == mk] ), mk))

24062 unique skills found in AU
26027 unique skills found in CA
10287 unique skills found in DE
5010 unique skills found in ES
2288 unique skills found in FR
31246 unique skills found in IN
4203 unique skills found in IT
3545 unique skills found in JP
1972 unique skills found in MX
34618 unique skills found in UK
55736 unique skills found in US


## 1.3 2021

### Load info from the 2021 json file

In [9]:
allf = getFiles( allyears[2] )
print(allf)

#NOTICE allo might contain repetitoins! (we filter them later)
allo = {}
for f in allf:
    allo[f] = loadObjects(f)
#list all markets
for key,value in allo.items():
    print(key)
    print(len(value))
    
#filter repetitoins and skills without name or developer
import json
year = '2021'

#Create a dictionary of unqiue skills per market
for f in allf:
    print('********************')
    market_name = f.split('/')[3]
    print(market_name)
    skillsdict = _createSkillDict(allo[f], market_name, year)
    print('#skills', len(skillsdict))
    
    skillsfound = []
    for sk in skillsdict:
        skill  = skillsdict[sk]
        skname = skill['name']
        skdev = skill['dev']      
     
    #save analysis in NewFullDataset json
    path = '../datasets/DatasetsProcessed/'+year+'/'+market_name+'.json'
    with open(path, 'w') as json_file:
        print('Saving json in ',path)
        json.dump(skillsdict, json_file) 

['../datasets/Dataset_Apri2021/AU/file.json', '../datasets/Dataset_Apri2021/CA/file.json', '../datasets/Dataset_Apri2021/DE/file.json', '../datasets/Dataset_Apri2021/ES/file.json', '../datasets/Dataset_Apri2021/FR/file.json', '../datasets/Dataset_Apri2021/IN/file.json', '../datasets/Dataset_Apri2021/IT/file.json', '../datasets/Dataset_Apri2021/JP/file.json', '../datasets/Dataset_Apri2021/MX/file.json', '../datasets/Dataset_Apri2021/UK/file.json', '../datasets/Dataset_Apri2021/US/file.json']
../datasets/Dataset_Apri2021/AU/file.json
24512
../datasets/Dataset_Apri2021/CA/file.json
27093
../datasets/Dataset_Apri2021/DE/file.json
10631
../datasets/Dataset_Apri2021/ES/file.json
5435
../datasets/Dataset_Apri2021/FR/file.json
2863
../datasets/Dataset_Apri2021/IN/file.json
28672
../datasets/Dataset_Apri2021/IT/file.json
4649
../datasets/Dataset_Apri2021/JP/file.json
3637
../datasets/Dataset_Apri2021/MX/file.json
2486
../datasets/Dataset_Apri2021/UK/file.json
37056
../datasets/Dataset_Apri2021/

### Create final dataset w/ traceability

In [10]:
#from excel
#test the join of the dataframes
year = '2021'
print('loading', excelpaths[2])
df2019 = loadExcelFile( excelpaths[2] )

#from json
datasetpath = "../datasets/DatasetsProcessed/"+year+"/"
skillsmarket = reloadData(datasetpath)

#join the two tables to create a general dataframe
finaldf = updateDataframe(skillsmarket, df2019)

#save dataframe
finaldf.to_csv('../datasets/DatasetsProcessedTraceability/'+year+'.csv')

loading ../datasets/Dataset_Apri2021/Dataset_April2021_traceability.csv
['../datasets/DatasetsProcessed/2021/AU.json', '../datasets/DatasetsProcessed/2021/CA.json', '../datasets/DatasetsProcessed/2021/DE.json', '../datasets/DatasetsProcessed/2021/ES.json', '../datasets/DatasetsProcessed/2021/FR.json', '../datasets/DatasetsProcessed/2021/IN.json', '../datasets/DatasetsProcessed/2021/IT.json', '../datasets/DatasetsProcessed/2021/JP.json', '../datasets/DatasetsProcessed/2021/MX.json', '../datasets/DatasetsProcessed/2021/UK.json', '../datasets/DatasetsProcessed/2021/US.json']

Total unique skills per market
AU :  24512
CA :  27093
DE :  10631
ES :  5435
FR :  2863
IN :  28672
IT :  4649
JP :  3637
MX :  2486
UK :  37056
US :  68667




### Some checks and analyses

In [18]:
finaldf = pd.read_csv('../datasets/DatasetsProcessedTraceability/2021.csv')

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
finaldf.columns

Index(['Unnamed: 0', 'id_new', 'id_aux', 'id', 'name', 'dev', 'cat', 'market',
       'subcat', 'acc_linking', 'in_skill_purchase', 'cost', 'review_stars',
       'skill_link', 'year', 'perm_requested_original', 'perm_found_norm',
       'policy_url', 'policy_text', 'traceability', 'perm_requested_norm'],
      dtype='object')

#### Total number of skills

In [20]:
allmarkets = finaldf.market.unique()
for mk in allmarkets:
    print('{} unique skills found in {}'.format(len(finaldf.loc[finaldf['market'] == mk] ), mk))

24512 unique skills found in AU
27093 unique skills found in CA
10631 unique skills found in DE
5435 unique skills found in ES
2863 unique skills found in FR
28672 unique skills found in IN
4649 unique skills found in IT
3637 unique skills found in JP
2486 unique skills found in MX
37056 unique skills found in UK
68667 unique skills found in US


In [26]:
finaldf.head()

,Unnamed: 0,id_aux,id,name,dev,cat,market,subcat,acc_linking,in_skill_purchase,cost,review_stars,skill_link,perm_requested_original,perm_found_norm,policy_url,policy_text,traceability,perm_requested_norm
0,0,Insureblocks Blockchain for insurance__by Insu...,B07GNDDNLG,Insureblocks Blockchain for insurance,by Insureblocks,Business & Finance,AU,Business & Finance,NaN,NaN,NaN,No customer reviews,https://www.amazon.com.au/Insureblocks-Blockch...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Bitcoin Portfolio Tracker__by Phocus Labs__B07...,B07C2TJPLV,Bitcoin Portfolio Tracker,by Phocus Labs,Business & Finance,AU,Business & Finance,NaN,NaN,NaN,No customer reviews,https://www.amazon.com.au/Phocus-Labs-Bitcoin-...,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Bag Facts__by Priya__B07H57WNTJ,B07H57WNTJ,Bag Facts,by Priya,Education & Reference,AU,Education & Reference,NaN,NaN,NaN,No customer reviews,https://www.amazon.com.au/Priya-Bag-Facts/dp/B...,NaN,NaN,NaN,NaN,NaN,NaN
3,3,My Baby Facts__by Right From Left Software__B0...,B071G3XDXH,My Baby Facts,by Right From Left Software,Education & Reference,AU,Education & Reference,NaN,NaN,NaN,No customer reviews,https://www.amazon.com.au/Right-From-Left-Soft...,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Bloomberg Market Minute__by Bloomberg L.P.__B0...,B07DVFN26J,Bloomberg Market Minute,by Bloomberg L.P.,Fitness & Sports,AU,News,NaN,NaN,NaN,No customer reviews,https://www.amazon.com.au/Bloomberg-L-P-Market...,NaN,NaN,NaN,NaN,NaN,NaN
